In [ ]:
data = gvol_client.portfolio_analyzer(portfolio= simulation
                                      ,deltaFutures=deltas_book
                                      ,ivShift= iv_shift
                                      , symbol= symbol)

book =pd.json_normalize(data['PortfolioAnalyzer'])
df = book
df['index'] = df['index'].astype(int)

In [ ]:
## Insert manually your equity
df['equity'] = (df['PnL'] + equity).round(4)
df['equityUSD'] = (df['equity'] * df['index']).astype(int)

In [ ]:
col = ['indexChange','PnL','PnLUSD','deltaBSM','deltaCash','deltaSkew','gamma','vega','wVega','theta','equity','equityUSD']


print ('')
print ('PORTFOLIO ANALYZER TABLE')
display(df[df['days']==0].iloc[50:115:5].set_index('index')[col]\
#display(df[df['days']==0].iloc[0:200:5].set_index('index')[col]\
.style.format({
            'indexChange':"{:20,.4f}",
            'PnL':"{:20,.4f}",
             'PnLUSD':"${:20,.0f}",
             'deltaBSM':"{:20,.2f}",
               'deltaCash':"{:20,.2f}",
               'deltaSkew':"{:20,.2f}",
             'gamma':"{:20.4f}",
              'vega':"{:20.0f}",
              'wVega':"{:20,.0f}",
            'theta':"{:20,.0f}",
            'index':"${:20,.0f}",
            'equity':"{:20,.4f}",
            'equityUSD':"${:20,.0f}",
            'days':"{:20,.0f}"

}))

In [ ]:
fig = go.Figure()
###CHART

index = df.iloc[80]['index']

minimo = 0
massimo = 0

for d in df.days.unique():
    df_t = df[df['days']==d]
    df_t = df_t.set_index('index')

    ##METRICS TO CHART
    metrics = ['PnL','PnLUSD','deltaSkew','gamma','vega','wVega','theta']
    ##IDENTIFY IN THE ARRAY ABOVE, THE POSITION OF METRICS TO CHART
    pos = 0

    x = df_t.index
    y_t = df_t[metrics[pos]]
    
    minimo = min(y_t.loc[index*0.7:index*1.3].min(),minimo)
    massimo = max(y_t.loc[index*0.7:index*1.3].max(),massimo)

    #prices = np.arange(index-index*0.45,index+index*0.75,100)
    prices = np.linspace(df[df['days']==0]['index'].min(),df[df['days']==0]['index'].max(),1000)

    if d in df.days.unique()[-1:]:
        z = interp1d(x, y_t,kind='linear')
        y_t_inter = z(prices)
        fig.add_trace(go.Scatter(x=prices, y=y_t_inter,name=metrics[pos]+' t'+str(int(d)),line_width=2,line=dict(dash='solid')))

    elif d not in df.days.unique()[-1:]:
        z = interp1d(x, y_t,kind='cubic')
        y_t_inter = z(prices)
        fig.add_trace(go.Scatter(x=prices, y=y_t_inter,name=metrics[pos]+' t'+str(int(d)),line_width=3,line=dict(dash='dot')))



    fig.add_vline(x=index, line_width=2, line_dash="dash", line_color="grey")
  


y_min =  minimo
y_max =  massimo

#fig.update_yaxes(range=[y_min,y_max])
#fig.update_xaxes(range=[index*0.7,index*1.3])

fig.layout.images = [dict(
    source="./amberdata_logo_bug_color_100p.png",
    xref="paper", yref="paper",
    x=0.98, y=1.1,
    sizex=0.13, sizey=0.13,
    xanchor="center", yanchor="bottom"
)]

fig.update_layout(title="PAYOFF STRUCTURE",title_x=0.50,
                  #yaxis_title="coin",
                  legend=dict(
                      orientation="h",
                      yanchor="bottom",
                      y=-0.2,
                      xanchor="center",
                      x=0.45
                      ),
                  margin=dict(r=25),
                 # template = 'plotly_dark'
                 )
fig.show()

In [ ]:
#DEFINE SUBPLOTS CHART
fig = make_subplots(rows=2,cols=2,subplot_titles=("delta", "gamma","vega","theta"),horizontal_spacing=0.05)

df[df['indexChange']==1]['index'].values[0]
#df[df['index_change']==1]['index'].values[0]

greeks = ['deltaBSM','gamma','vega','theta']
#greeks = ['delta_BSM','gamma','vega','theta']

row = 1
col = 1
inc_row = [0,1,0,0]
inc_col = [1,-1,1,0]

colore = ['blue','red','green']


for pos in range(0,len(greeks)):
    i = 0
    
    minimo = 0
    massimo = 0

    for d in df.days.unique()[:-1]:
        df_t = df[df['days']==d]
        df_t = df_t.set_index('index')
 

        x = df_t.index
        y_t = df_t[greeks[pos]]


        minimo = min(y_t.loc[index*0.7:index*1.3].min(),minimo)
        massimo = max(y_t.loc[index*0.7:index*1.3].max(),massimo)
        
        prices = np.linspace(df[df['days']==0]['index'].min(),df[df['days']==0]['index'].max(),1000)

        if d in df.days.unique()[-1:]:
            z = interp1d(x, y_t,kind='linear')
            y_t_inter = z(prices)
            fig.add_trace(go.Scatter(x=prices, y=y_t_inter,name=greeks[pos]+' t'+str(int(d)),line_width=2,line=dict(dash='solid',color=colore[i])),
                         row = row, col = col)

        elif d not in df.days.unique()[-1:]:
            z = interp1d(x, y_t,kind='cubic')
            y_t_inter = z(prices)
            fig.add_trace(go.Scatter(x=prices, y=y_t_inter,name=greeks[pos]+' t'+str(int(d)),line_width=3,line=dict(dash='dot',color=colore[i])),
                         row = row, col = col)



        fig.add_vline(x=index, line_width=2, line_dash="dash", line_color="grey",row=row, col=col)
  
        i = i + 1
    
    y_min =  minimo
    y_max =  massimo

    fig.update_yaxes(range=[y_min,y_max],row=row,col=col)
    fig.update_xaxes(range=[index*0.7,index*1.3],row=row,col=col)
    
    row = row + inc_row[pos]
    col = col + inc_col[pos]
            
fig.layout.images = [dict(
    source="./amberdata_logo_bug_color_100p.png",
    xref="paper", yref="paper",
    x=0.98, y=1.1,
    sizex=0.13, sizey=0.13,
    xanchor="center", yanchor="bottom"
)]

fig.update_layout(title="GREEKS",title_x=0.50,
                  legend=dict(
                      orientation="h",
                      yanchor="bottom",
                      y=-0.2,
                      xanchor="center",
                      x=0.45
                      ),
                  margin=dict(r=25),
                 # template = 'plotly_dark'
                 )

fig.update_layout(showlegend=False)    
    
    
fig.show()